In [1]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
def QuantizeColor(img, n=64):
    div = 256//n
    rgb = cv2.split(img)
    q = []
    for ch in rgb:
        vf = np.vectorize(lambda x, div: int(x//div)*div)
        quantized = vf(ch, div)
        q.append(quantized.astype(np.uint8))
    d_img = cv2.merge(q)
    return d_img

def ccv(src, tau=0, n=64):
    img = src.copy()
    row, col, channels = img.shape
    #print col
    if not col == 300:
        aspect = float(300)/float(col)
        #print aspect
        img = cv2.resize(img, None, fx=aspect, fy=aspect, interpolation = cv2.INTER_CUBIC)
    row, col, channels = img.shape
    # blur
    img = cv2.GaussianBlur(img, (3,3),0)
    # quantize color
    img = QuantizeColor(img, n)
    bgr = cv2.split(img)
    if tau == 0:
        tau = row*col * 0.1
    alpha = np.zeros(n)
    beta = np.zeros(n)
    # labeling
    for i,ch in enumerate(bgr):
        ret,th = cv2.threshold(ch,127,255,0)
        ret, labeled, stat, centroids = cv2.connectedComponentsWithStats(th, None, cv2.CC_STAT_AREA, None, connectivity=8)
    # generate ccv
        areas = [[v[4],label_idx] for label_idx,v in enumerate(stat)]
        coord = [[v[0],v[1]] for label_idx,v in enumerate(stat)]
    # Counting
        for a,c in zip(areas,coord):
            area_size = a[0]
            x,y = c[0], c[1]
            if (x < ch.shape[1]) and (y < ch.shape[0]):
                bin_idx = int(float(ch[y,x])/float(256/float(n)))
                if area_size >= tau:
                    alpha[bin_idx] = alpha[bin_idx] + area_size
                else:
                    beta[bin_idx] = beta[bin_idx] + area_size
    return alpha, beta



def extract_feature(img):
    frame = img
    roiWid = 10
    #roiEdg = 8
    roiEdg = 10
    src_height, src_width, src_channels = frame.shape
    roiX = int(src_width / roiWid)
    roiWidth = roiX * roiEdg
    roiY = int(src_height / roiWid)
    roiHeight = roiY * roiEdg
    RoiImg = frame[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
    
    n=128
    alpha, beta = ccv(RoiImg, tau=0,n=n)
    CCV = alpha.tolist()+beta.tolist()
    X = [x for x in range(n*2)]
    Y = alpha.tolist()+beta.tolist()
    
    
    HSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    H,S,V = cv2.split(HSV)
    Vue_STD = np.std(V)
    Hue_STD = np.std(H)
    Hue_mean = np.mean(H)
    Gimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    G_STD = np.std(Gimg)
    hist,bins = np.histogram(img.flatten(),256,[0,256])
    hist2 = cv2.calcHist([img],[0],None,[256],[0,256])
    hist_STD = np.std(hist)
    hist_Mean = np.mean(hist)
    r = img[...,2]
    g = img[...,1]
    b = img[...,0]
    rY = 0.212655;
    gY = 0.715158;
    bY = 0.072187;
    inner = RoiImg
    ir = inner[...,2]
    ig = inner[...,1]
    ib = inner[...,0]

    Luminance = r*0.2126+g*0.7152+0.0722*b
    Li = ir*0.2126+ig*0.7152+0.0722*ib
    hei,wid,chan = img.shape
    ih,iw,ic = inner.shape
    Lum_value = (np.sum(Luminance)-np.sum(Li))/((hei*wid)-(ih*iw))
    
    mr = np.sqrt((np.mean(r)-119)*(np.mean(r)-119))
    mg = np.sqrt((np.mean(g)-119)*(np.mean(g)-119))
    mb = np.sqrt((np.mean(b)-119)*(np.mean(b)-119))
    GW_value = (mr+mg+mb)/3
    
    
    
    return [Vue_STD,Hue_STD,Hue_mean,G_STD,GW_value,np.median(hist),Y]

In [61]:
import pickle
GT_map = pickle.load(open('color_dataset.pkl','r'))
print sum(sum(GT_map.values()))

67958


In [62]:
#ROOT = '/root/ND_classifier/color_data/Vehicle_color_49466'
import pickle
#features = pickle.load(open('feautres.pkl','r'))
#X_test = pickle.load(open('X_test.pkl','r'))
features = []
X_test = []
labels = []
labels_test = []
color_set = ['red','yellow','blue','green','white','black','gray']
GT_map = pickle.load(open('color_dataset.pkl','r'))

for key,val in GT_map.iteritems():
    #print key,val
    #print val.astype(float)
    #if np.random.rand() < 0.9 :
    #    continue
    im = cv2.imread(key)
    out = extract_feature(im)
    if (np.random.rand()<0.8):
        features.append(out[0:6]+out[6])
        labels.append(val.astype(float))
    else:
        X_test.append(out[0:6]+out[6])
        labels_test.append(val.astype(float))

In [63]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

classifier = DecisionTreeClassifier(max_depth=5)
classifier.fit(features,labels)

Y = classifier.predict(X_test)



print Y
#Y = [i.tolist() for i in Y]
Y_ = [i.tolist() for i in labels_test]
#print classifier.score(X_test,Y_)
#print tuple(labels_test)
print Y_
print np.asarray(labels_test)
accuracy = metrics.f1_score(Y_,Y)

[[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0

ValueError: Classification metrics can't handle a mix of multiclass-multioutput and multilabel-indicator targets

In [64]:
import pickle
from sklearn.externals import joblib
#joblib.dump(regr_2, 'depth5.pkl')


file = open('feautres.pkl', 'wb')
pickle.dump(features, file)
file.close()
file = open('labels.pkl','wb')
pickle.dump(labels,file)
file.close()
file = open('X_test.pkl', 'wb')
pickle.dump(X_test, file)
file.close()
file = open('labels_test.pkl', 'wb')
pickle.dump(labels_test, file)
file.close()

In [ ]:
file = open('feautres.pkl', 'wb')
pickle.dump(features, file)
file.close()
file = open('labels.pkl','wb')
pickle.dump(labels,file)
file.close()
file = open('X_test.pkl', 'wb')
pickle.dump(X_test, file)
file.close()
file = open('labels_test.pkl', 'wb')
pickle.dump(labels_test, file)
file.close()

In [96]:
#import itertools
img_src = '/root/ND_classifier/color_data/Vehicle_color_49466/2_white/van18.53.37.jpg_1.jpg'
im = cv2.imread(img_src)
out = extract_feature(im)
#print out
#print out[0:6]+out[6]
#print features
color_set = ['red','yellow','blue','green','white','black','gray']
label = 'red'
input_feature = out[0:6]+out[6]
#print np.nonzero(classifier.predict([input_feature])[0])[0][0]
Y = classifier.predict(X_test)
Y_ = np.asarray([i.tolist() for i in labels_test])
#print Y
#print Y_



13354


In [133]:
from sklearn import preprocessing


TP = [0,0,0,0,0,0,0]
FP = [0,0,0,0,0,0,0]
TN = [0,0,0,0,0,0,0]
FN = [0,0,0,0,0,0,0]
GT = [0,0,0,0,0,0,0]
data_points = 49466
colors = ['red','yellow','blue','green','white','black','gray']

parameter = "criterion='gini',max_depth=None,min_samples_split=10,min_samples_leaf=5"

X = features

classifier = DecisionTreeClassifier(criterion='gini',max_depth=None,min_samples_split=10,min_samples_leaf=5)
classifier.fit(X,labels)

Y = classifier.predict(X_test)
Y_ = np.asarray([i.tolist() for i in labels_test])

for i in range(0,len(Y)):
    Y_i = Y_[i]
    Yi = Y[i]
    for j in range(0,7):
        if (Yi[j]==Y_i[j]):
            if (Y_i[j]==1):
                TP[j] +=1
                GT[j] +=1
            else:
                TN[j] +=1
        else:
            if (Y_i[j]==1):
                FN[j] +=1
                GT[j] +=1
            else:
                FP[j] +=1

print GT
precision = []
recall = []
accuracy = []
param = []


for i in range(0,7) :
    Tp = float(TP[i])
    Tn = float(TN[i])
    Fp = float(FP[i])
    Fn = float(FN[i])
    precision.append(Tp/(Tp+Fp+1))
    accuracy.append((Tp+Tn)/(Tp+Tn+Fp+Fn))
    recall.append(Tp/(Tp+Fn+1))

print "Precision = " ,["%.4f" % p for p in precision]
print "Recall = ",["%.4f" % r for r in recall]
print "Accuracy = ",["%.4f" % a for a in accuracy]


param.append(parameter) 
allp.append(precision)
allr.append(recall)
alla.append(accuracy)



[546, 126, 472, 5391, 2716, 2597, 1666]
Precision =  ['0.8197', '0.3830', '0.5084', '0.9551', '0.8134', '0.7782', '0.5291']
Recall =  ['0.7148', '0.1417', '0.3192', '0.9427', '0.7957', '0.7510', '0.4421']
Accuracy =  ['0.9820', '0.9898', '0.9651', '0.9591', '0.9214', '0.9101', '0.8814']


In [127]:
print "Precision = " ,["%.4f" % p for p in allp[2]]
print "Recall = ",["%.4f" % r for r in allr[2]]
print "Accuracy = ",["%.4f" % a for a in alla[2]]



Precision =  ['0.8012', '0.3167', '0.4984', '0.9515', '0.8074', '0.7738', '0.5369']
Recall =  ['0.7294', '0.1496', '0.3277', '0.9431', '0.7792', '0.7425', '0.4757']
Accuracy =  ['0.9817', '0.9890', '0.9647', '0.9578', '0.9174', '0.9078', '0.8835']


In [115]:
training_GT = [0,0,0,0,0,0,0]
for i in range(0,len(labels)):
    label = labels[i]
    for j in range(0,7):
        if (label[j] !=0):
            training_GT[j] +=1

print training_GT

[2270, 437, 1846, 21673, 11477, 10197, 6544]


In [87]:
ROOT = '/root/ND_classifier/color_data/Vehicle_color_49466'
TP = [0,0,0,0,0,0,0]
FP = [0,0,0,0,0,0,0]
#TN = [49466,49466,49466,49466,49466,49466,49466]
#FN = [49466,49466,49466,49466,49466,49466,49466]
TN = [0,0,0,0,0,0,0]
FN = [0,0,0,0,0,0,0]
GT = [0,0,0,0,0,0,0]
data_points = 49466
colors = ['red','yellow','blue','green','white','black','gray']

    

for root, subroot, fnames in os.walk(ROOT):
    for fname in fnames:
        if (np.random.rand()<0.8):
            continue
        target = os.path.join(root,fname)
        label = target.split("/")[5].split("_")[1]
        #if (int(target.split("/")[5].split("_")[0])>17):
        #    continue
        #print target.endswith('.jpg')
        if (target.endswith('.jpg')==False):
            continue
        print target , label
        im = cv2.imread(target)
        out = extract_feature(im)
        feature = out[0:6]+out[6]
        #search_color = Search_Color(auto_update_status = False)
        #res = search_color.color_detection_test(im)
        res = colors[np.nonzero(classifier.predict([input_feature])[0])[0][0]]
        GT[colors.index(label)]+=1
        for color in colors:
            if (color == label):
                if(label == res):
                    TP[colors.index(color)]+=1
                else:
                    FN[colors.index(color)]+=1
            else:
                if(color == res):
                    FP[colors.index(color)]+=1
                else:
                    TN[colors.index(color)]+=1

/root/ND_classifier/color_data/Vehicle_color_49466/2_white/sedan_suv18.15.41.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/2_white/sedan_suv18.03.27.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/2_white/sedan_suv18.53.31.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/2_white/van18.53.40.jpg_5.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/2_white/sedan_suv18.53.34.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/2_white/sedan_suv18.13.36.jpg_4.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/2_white/sedan_suv18.04.46.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/2_white/sedan_suv18.06.09.jpg_3.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/2_white/sedan_suv18.53.30.jpg_2.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/2_white/sedan_suv18.53.37.jpg_4.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/2_white/van18.53.38.jpg_1

/root/ND_classifier/color_data/Vehicle_color_49466/14_red/truck06.13.51.jpg_4.jpg red
/root/ND_classifier/color_data/Vehicle_color_49466/14_red/van06.34.14.jpg_2.jpg red
/root/ND_classifier/color_data/Vehicle_color_49466/14_red/sedan_suv06.12.19.jpg_3.jpg red
/root/ND_classifier/color_data/Vehicle_color_49466/14_red/sedan_suv06.17.37.jpg_2.jpg red
/root/ND_classifier/color_data/Vehicle_color_49466/14_red/sedan_suv06.24.30.jpg_1.jpg red
/root/ND_classifier/color_data/Vehicle_color_49466/5_white/truck21.37.32.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/5_white/truck21.38.11.jpg_2.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/5_white/van21.34.09.jpg_2.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/5_white/truck21.37.39.jpg_2.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/5_white/sedan_suv21.34.13.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/5_white/sedan_suv21.33.19.jpg_1.jpg white
/root/ND_classifier/

/root/ND_classifier/color_data/Vehicle_color_49466/10_yellow/van02.32.35.jpg_2.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/10_yellow/sedan_suv02.30.02.jpg_1.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/10_yellow/sedan_suv02.37.38.jpg_2.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/10_yellow/sedan_suv02.55.10.jpg_2.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/10_yellow/sedan_suv02.47.41.jpg_2.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/10_yellow/van02.23.57.jpg_1.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/10_yellow/sedan_suv02.32.33.jpg_2.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/10_yellow/sedan_suv02.32.51.jpg_2.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/10_yellow/sedan_suv02.20.25.jpg_2.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/10_yellow/sedan_suv02.55.14.jpg_2.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/1

/root/ND_classifier/color_data/Vehicle_color_49466/19_white/sedan_suv11.41.36.jpg_5.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/19_white/sedan_suv11.33.31.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/19_white/sedan_suv11.31.42.jpg_3.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/19_white/van11.43.30.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/19_white/sedan_suv11.34.14.jpg_10.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/19_white/sedan_suv11.23.41.jpg_3.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/19_white/sedan_suv11.17.17.jpg_8.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/19_white/van11.07.09.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/19_white/van11.00.09.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/19_white/sedan_suv11.55.16.jpg_2.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/19_white/sedan_suv11.37.21

/root/ND_classifier/color_data/Vehicle_color_49466/20_black/sedan_suv12.00.33.jpg_10.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/20_black/sedan_suv12.06.53.jpg_6.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/20_black/sedan_suv12.00.25.jpg_3.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/20_black/van12.26.41.jpg_2.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/20_black/sedan_suv12.00.21.jpg_6.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/20_black/sedan_suv12.16.39.jpg_4.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/20_black/sedan_suv12.50.02.jpg_4.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/20_black/sedan_suv12.00.21.jpg_3.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/20_black/sedan_suv12.06.55.jpg_4.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/20_black/sedan_suv12.30.31.jpg_5.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/20_black/sedan

...

/root/ND_classifier/color_data/Vehicle_color_49466/22_yellow/sedan_suv14.37.34.jpg_5.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/22_yellow/sedan_suv14.11.34.jpg_1.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/22_yellow/sedan_suv14.51.26.jpg_2.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/22_yellow/sedan_suv14.43.00.jpg_3.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/22_yellow/sedan_suv14.31.32.jpg_2.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/22_yellow/sedan_suv14.12.45.jpg_3.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/22_yellow/sedan_suv14.13.03.jpg_2.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/22_yellow/sedan_suv14.40.15.jpg_3.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/22_yellow/sedan_suv14.51.42.jpg_2.jpg yellow
/root/ND_classifier/color_data/Vehicle_color_49466/22_yellow/sedan_suv14.51.25.jpg_1.jpg yellow
/root/ND_classifier/color_data/Vehicle_c

/root/ND_classifier/color_data/Vehicle_color_49466/19_gray/sedan_suv11.50.15.jpg_6.jpg gray
/root/ND_classifier/color_data/Vehicle_color_49466/19_gray/sedan_suv11.21.18.jpg_6.jpg gray
/root/ND_classifier/color_data/Vehicle_color_49466/19_gray/sedan_suv11.00.10.jpg_2.jpg gray
/root/ND_classifier/color_data/Vehicle_color_49466/19_gray/sedan_suv11.32.03.jpg_1.jpg gray
/root/ND_classifier/color_data/Vehicle_color_49466/19_gray/sedan_suv11.24.12.jpg_6.jpg gray
/root/ND_classifier/color_data/Vehicle_color_49466/19_gray/sedan_suv11.37.03.jpg_5.jpg gray
/root/ND_classifier/color_data/Vehicle_color_49466/19_gray/sedan_suv11.33.06.jpg_8.jpg gray
/root/ND_classifier/color_data/Vehicle_color_49466/19_gray/van11.20.07.jpg_1.jpg gray
/root/ND_classifier/color_data/Vehicle_color_49466/19_gray/sedan_suv11.47.07.jpg_15.jpg gray
/root/ND_classifier/color_data/Vehicle_color_49466/19_gray/sedan_suv11.48.08.jpg_7.jpg gray
/root/ND_classifier/color_data/Vehicle_color_49466/19_gray/sedan_suv11.47.07.jpg_11.j

/root/ND_classifier/color_data/Vehicle_color_49466/10_white/sedan_suv02.35.40.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/10_white/sedan_suv02.47.38.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/10_white/van02.33.56.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/10_white/sedan_suv02.10.03.jpg_2.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/10_white/sedan_suv02.43.25.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/10_white/sedan_suv02.57.30.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/10_white/sedan_suv02.17.38.jpg_5.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/10_white/van02.01.29.jpg_1.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/12_white/van04.50.09.jpg_2.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/12_white/sedan_suv04.40.46.jpg_2.jpg white
/root/ND_classifier/color_data/Vehicle_color_49466/12_white/sedan_suv04.08.17.

/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.44.20.jpg_1.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.28.45.jpg_1.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.55.05.jpg_4.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.50.14.jpg_6.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.44.25.jpg_1.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.50.30.jpg_3.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.43.43.jpg_5.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.13.49.jpg_3.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.48.39.jpg_1.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.40.45.jpg_5.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/

/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.27.17.jpg_5.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.56.57.jpg_5.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.03.25.jpg_2.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.54.36.jpg_4.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.28.44.jpg_1.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.25.18.jpg_7.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.40.38.jpg_6.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.37.41.jpg_3.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.28.46.jpg_1.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/sedan_suv06.40.40.jpg_6.jpg black
/root/ND_classifier/color_data/Vehicle_color_49466/14_black/

In [100]:
print colors
#print TP
#print FP
#print TN
#print FN
print GT
precision = []
recall = []
accuracy = []
for i in range(0,7) :
    Tp = float(TP[i])
    Tn = float(TN[i])
    Fp = float(FP[i])
    Fn = float(FN[i])
    precision.append(Tp/(Tp+Fp+1))
    accuracy.append((Tp+Tn)/(Tp+Tn+Fp+Fn))
    recall.append(Tp/(Tp+Fn+1))
print "Precision = " ,["%.4f" % p for p in precision]
print "Recall = ",["%.4f" % r for r in recall]
print "Accuracy = ",["%.4f" % a for a in accuracy]

['red', 'yellow', 'blue', 'green', 'white', 'black', 'gray']
[546, 126, 472, 5391, 2716, 2597, 1666]
Precision =  ['0.7333', '0.0000', '0.0000', '0.9018', '0.7863', '0.7566', '0.6667']
Recall =  ['0.3821', '0.0000', '0.0000', '0.9301', '0.7324', '0.6570', '0.0216']
Accuracy =  ['0.9691', '0.9906', '0.9647', '0.9310', '0.9052', '0.8923', '0.8767']


In [ ]:
['red', 'yellow', 'blue', 'green', 'white', 'black', 'gray']
[27, 5149, 0, 0, 1132, 1883, 501]
[6, 473, 0, 0, 225, 361, 305]
[9951, 4287, 9926, 10057, 8346, 7660, 8775]
[78, 153, 136, 5, 359, 158, 481]
[105, 5302, 136, 5, 1491, 2041, 982]
Precision =  ['0.7941', '0.9157', '0.0000', '0.0000', '0.8336', '0.8388', '0.6208']
Recall =  ['0.2547', '0.9710', '0.0000', '0.0000', '0.7587', '0.9221', '0.5097']
Accuracy =  ['0.9917', '0.9378', '0.9865', '0.9995', '0.9420', '0.9484', '0.9219']